Define functions for cropping images

In [ ]:
def crop_bottom(image, percent):
    height, width = image.shape[:2]
    cutoff = int(height * (1 - percent))
    cropped_image = image[:cutoff, :]
    return cropped_image

def crop_top(image, percent):
    height, width = image.shape[:2]
    cutoff = int(height * percent)
    cropped_image = image[cutoff:, :]
    return cropped_image

def crop_lr(image, percent):
    height, width = image.shape[:2]
    crop_width = int(width * percent)
    cropped_image = image[:, crop_width:-crop_width]
    return cropped_image

### The Main PaddleOCR Loop

The following code block loops over the SoccerNet challenge tracklets and makes predictions for each image. Make sure the path to the data is correct for your computer. The predictions for the tracklet are stored in the 'jersey' array if they surpass the confidence threshold (otherwise they're assigned -1). A majority vote is used to determine the overall prediction for the tracklet. There are some image processing steps prior to PaddleOCR detection/recognition: cropping, rescaling, gaussian blur, grayscale, contrast. The tracklet folder indices and their corresponding labels are stored in the 'folder' and 'predictions' arrays, respectively. Note: you may encounter an indexing error for extracting the prediction tuple (I didn't have this issue but someone on my team did)

In [ ]:
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR
from collections import Counter
import cv2, os

# Initialize PaddleOCR with the English language
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log=False)

predictions = []
folder = []

# Loop over all 1425 challenge tracklets
for i in range(0,1426):

    # Path to the folder containing images
    folder_path = f'/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/SoccerNet/jersey-2023/challenge/images/{i}/'

    # Get a list of all files in the folder
    image_files = os.listdir(folder_path)

    jersey_numbers = []

    # Iterate over each image file in the folder
    for file_name in image_files:
        # Construct the full path to the image file
        image_path = os.path.join(folder_path, file_name)
        
        # Read the image using OpenCV
        img = cv2.imread(image_path)


        # Crop
        img = crop_bottom(img, 0.5)
        img = crop_top(img, 0.1)
        img = crop_lr(img, 0.1)

        height, width = img.shape[:2]

        # Make sure image has non-zero dimension to prevent error
        if height < 1 or width < 1:
            continue

        # Rescale the image by factor 'scale'
        scale = 3.0
        img = cv2.resize(img, (round(scale * width), round(scale * height)), interpolation=cv2.INTER_CUBIC)
        
        # Reduce noise with a Gaussian blur
        img = cv2.GaussianBlur(img, (5, 5), 0)

        # Convert to grayscale
        #if len(img.shape) == 3:  # If image is color (has 3 channels)
        #    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply contrast
        #alpha = 2  # Contrast control
        #beta = 0   # Brightness control
        #img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

        
        # Perform OCR on the image
        result = ocr.ocr(img, cls=True)
        
        # Check if the list is empty
        if result:
            # Extract the tuple ('13', 0.9994895458221436)
            result = result[0][1]
            # Only count numeric results (exclude detected letters)
            if result[0].isdigit():
                # Confidence threshold is 0.6
                if result[1] > 0.6:
                    jersey_numbers.append(int(result[0]))

    print(jersey_numbers)

    # Count occurrences of each value
    counts = Counter(jersey_numbers)

    # Majority vote (most common detected value is the jersey number)
    if counts:
        most_common_value = counts.most_common(1)[0][0]
    else:
        most_common_value = -1

    print('Folder:',str(i),'Jersey:',str(most_common_value))
    predictions.append(most_common_value)
    folder.append(i)


### Output results to JSON

The following code block writes the results to a JSON dictionary file in the same format as the provided JSON files for the training and test data.

In [ ]:
import json 

# Zip arrays A and B to create a list of tuples
key_value_pairs = zip(folder,predictions)

# Convert the list of tuples into a dictionary
json_dict = dict(key_value_pairs)

# Write the dictionary to a JSON file
with open('output.json', 'w') as json_file:
    json.dump(json_dict, json_file)

The next code block allows you to compare your JSON to the ground truth JSON. This was useful for evaluating the performance of our model on the test data.

In [ ]:
def compare_json_files(file1, file2):
    with open(file1, 'r') as f1:
        data1 = json.load(f1)

    with open(file2, 'r') as f2:
        data2 = json.load(f2)

    total_pairs_b = len(data2)
    matching_pairs = 0

    for key, value in data2.items():
        if key in data1 and data1[key] == value:
            matching_pairs += 1

    if total_pairs_b > 0:
        similarity_percentage = (matching_pairs / total_pairs_b) * 100
    else:
        similarity_percentage = 0

    return similarity_percentage

# Usage
file1 = "test_gt.json"
file2 = "output.json"
similarity_percentage = compare_json_files(file1, file2)
print("Accuracy of the model (% of correct predictions):\n", similarity_percentage)

### Prediction Evaluations with a Confusion Matrix

This codeblock defines functions for plotting a confusion matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

def extract_values(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return list(data.values())

def plot_confusion_matrix(true_file, pred_file):
    # Extract true values and predictions from JSON files
    true_values = extract_values(true_file)
    predictions = extract_values(pred_file)

    #true_values = true_values[0:100]

    # Get unique labels
    labels = np.unique(true_values + predictions)

    # Calculate confusion matrix
    cm = confusion_matrix(true_values, predictions, labels=labels)

    # Normalize confusion matrix
    cm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 0.01)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=False, fmt=".2f", cmap="viridis", xticklabels=labels, yticklabels=labels, linewidths=.5, linecolor='gray')
    plt.xlabel('Predicted', fontsize=24)  # Adjust fontsize as needed
    plt.ylabel('True', fontsize=24)       # Adjust fontsize as needed
    plt.title('Normalized Confusion Matrix, batch size 32 (color)', fontsize=24)
    plt.show()

This block plots the confusion matrix for two JSON files.

In [ ]:
true_file = 'test_gt.json'
pred_file = 'output.json'
plot_confusion_matrix(true_file, pred_file)

### The Multipass Approach

The multipass approach we used for our challenge submission was accomplished by running the main loop multiple times with different image processing parameters. Make sure to save all the JSON files for each pass. One could write a script to automatically use them to aggregate results as shown in Figure 7 of our report. We instead manually aggregated results in Excel. If you have further requests for code or explanations, or if you have issues running the code, please contact zcroft@umich.edu.